## Phase 0: Import basic modules

In [1]:
# Import modules
import os
import subprocess
import fileinput
from pathlib import Path
from shutil import copyfile

# PHASE 1: Set up workspace # 
## STEP 1: Install required programs: MAFFT, pal2nal, IQ-TREE and PAML ##

#### MAFFT must be downloaded at https://mafft.cbrc.jp/alignment/software/ (available for macOS, Windows, and Linux)
#### After downloading, run:

In [346]:
from Bio.Align.Applications import MafftCommandline # Imports MafftCommandline wrapper module for MAFFT

#### Save your pathway here:

In [347]:
mafftfolder = Path("/opt/anaconda3")     # Assigns folder for MAFFT to path variable
MafftExec = mafftfolder / "bin/mafft"    # Assigns mafft executable to variable based on provided directory
#           your subdir + MAFFT installation subdir

#### pal2nal can be installed via bioconda or downloaded at: http://www.bork.embl.de/pal2nal/

In [229]:
conda install -c bioconda pal2nal # Installs pal2nal via bioconda

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


#### Either way, save your pathway here:

In [348]:
pal2nalfolder = Path("/opt/anaconda3/pkgs/pal2nal-14.1-pl526_1") # Assigns folder for pal2nal to path variable
pal2nalExec = pal2nalfolder / "bin/pal2nal.pl"                   # Assigns pal2nal executable to variable based on provided directory
#               your subdir + pal2nal installation subdir

#### IQ-TREE must be downloaded at http://www.iqtree.org/ (available for macOS, Windows, and Linux)
#### Enter the path to your local instance of IQ-Tree in the quotes below:
##### (If you did not install IQ-TREE in your working directory as I did, remove os.getcwd() from the code below and store the full path as iqtreefolder instead.)

In [349]:
iqtreefolder = Path("iqtree-1.6.12-MacOSX")        # Assigns folder for IQ-Tree to variable
IQExec = os.getcwd() / iqtreefolder / "bin/iqtree" # Assigns IQ-Tree executable to variable based on provided directory
#  working directory + your subdir + iqtree subdir

#### PAML must first be downloaded at http://abacus.gene.ucl.ac.uk/software/paml.html (available for MacOSX, Windows, and UNIX/Linux)
#### Enter the path to your local instance of PAML in the quotes below:
##### (If you did not install PAML in your working directory as I did, remove os.getcwd() from the code below and store the full path as pamlfolder instead.)

In [350]:
pamlfolder = Path("paml4.8")                            # Assigns folder for PAML to variable
PAMLExec = os.getcwd() / pamlfolder / "bin/codeml"      # Assigns PAML codeml executable to variable based on provided directory
#   working directory + your subdir + paml subdir
PAMLcodemlCtl = os.getcwd() / pamlfolder / "codeml.ctl" # Assigns PAML codeml control file full path to variable based on provided directory
#        working directory + your subdir + filename

## STEP 2: Define datasources ##
#### Enter the path for your local data folders in the quotes below:

In [351]:
pepfolder = Path("pep_from_LegInfo")  # Assigns folder (mine is in my working directory) containing all your peptides to variable
pepfix = ".pep.fa"                    # Specifies suffix for peptide files
nucfolder = Path("nuc_from_LegInfo")  # Assigns folder (mine is in my working directory) containing all your nucleotides to variable
nucfix = ".nuc.fa"                    # Specifies suffix for nucleotide files
treefoldername = "trees"              # Specifies name for output tree folder
codemlfoldername = "codeml"           # Specifies name for output codeml folder
treefolder = Path(treefoldername)     # Assigns tree folder (mine is in my working directory) to variable
codemlfolder = Path(codemlfoldername) # Assigns codeml folder (mine is in my working directory) to variable

## STEP 3: Check that datasources and local processes exist: ##

In [352]:
if not mafftExec.exists():                                     # Checks if MAFFT installed
    print("MAFFT not found, terminating.")                     # Reports any negative
    exit()                                                     # Terminates
elif not pal2nalExec.exists():                                 # Checks if pal2nal installed
    print("pal2nal not found, terminating.")                   # Reports any negative
    exit()                                                     # Terminates
elif not IQExec.exists():                                      # Checks if IQ-TREE installed
    print("IQ-TREE not found, terminating.")                   # Reports any negative
    exit()                                                     # Terminates
elif not PAMLExec.exists():                                    # Checks if PAML installed
    print("PAML not found, terminating.")                      # Reports any negative
    exit()                                                     # Terminates
elif not pepfolder.exists():                                   # Checks if peptide files specified
    print("Peptide folder not found, terminating.")            # Reports any negative
    exit()                                                     # Terminates
elif not nucfolder.exists():                                   # Checks if nucleotide files specified
    print("Nucleotide folder not found, terminating.")         # Reports any negative
    exit()                                                     # Terminates
else:                                                          # 
    print("All files and executables found, proceeding...")    # Reports check passing
    
if not treefolder.exists():                                    # Checks if tree folder specified
    os.mkdir(treefoldername)                                   # Generates tree folder if not
    treefolder = Path(treefoldername)                          # Assigns just-generated tree folder to variable above
if not codemlfolder.exists():                                  # Checks if codeml folder specified
    os.mkdir(codemlfoldername)                                 # Generates codeml folder if not
    codemlfolder = Path(codemlfoldername)                      # Assigns just-generated codeml folder to variable above

All files and executables found, proceeding...


# PHASE 2: Do alignments #
## STEP 1: Align protein sequences with MAFFT ##
##### (Note: you can modify the alignment output suffix at: alnfix = ".[enter suffix here]")

In [353]:
# Loop over all files in peptide folder
for filename in os.listdir(pepfolder):                # specify the peptide folder as the one to look for files in
    if filename.endswith(pepfix):                     # make sure that only peptide fastas are used
        
        infile = pepfolder / filename                 # specify full input peptide fasta path
        alnfix = ".aligned"                           # specify suffix for output
        outfile = pepfolder / (filename + alnfix)     # generate output peptide alignment path
        
        mafft_cline = MafftCommandline(input=infile)  # generate mafft command
        stdout, stderr = mafft_cline()                # run mafft command, save to stdout & stderr
        
        with open(outfile, "w") as handle:            # specify output file path as save destination
            handle.write(stdout)                      # save stdout to output file path
        continue
    else:
        continue

## STEP 2: Align codon sequences to protein alignments with pal2nal ##

In [354]:
# Store alignment suffix as a variable
pepalnfix = (pepfix + alnfix)

# Loop over all files in peptide folder
for filename in os.listdir(pepfolder):                # specify the peptide folder as the one to look for files in
    if filename.endswith(pepalnfix):                  # make sure that only peptide alignments are used
        
        nucname = filename.replace(pepalnfix, nucfix) # generate the basename of peptide alignment's corresponding nucleotide fasta
        pepfile = pepfolder / filename                # specify full input peptide alignment path
        nucfile = nucfolder / nucname                 # specify full input nucleotide fasta path
        outfile = nucfolder / (nucname + alnfix)      # specify output nucleotide alignment path
        
        stdout = subprocess.run(["perl", str(pal2nalExec), str(pepfile), str(nucfile), "-nogap", "-output", "fasta"], capture_output=True).stdout.decode("utf-8")
                                                      # generate and run command, save as stdout
                #          COMMAND CONTEXT STRUCTURE: #
                # subprocess.run(...)                 # This part tells python it's going to run some command-line-style command
                # ([command], ...)                    # This part specifies the command as a list of arguments
                # (..., capture_output=True)          # This part stores the output as bytes in the stdout built-in field of the subprocess module
                # ...(...).stdout...                  # This part retrieves that byte output from subprocess' stdout field
                # ...(...)...decode("utf-8")          # This part decodes the byte output into a string
                # stdout = subprocess.run(...)...     # This part stores that string as a python variable
                #                  COMMAND STRUCTURE: #
                # "perl"                              # This part tells python it's going to be running perl
                # str(pal2nalExec)                    # This part passes the string name of the pal2nal exectauble path as an arg to perl
                # str(pepfile)                        # This part passes the string name of the current peptide alignment file as an arg to pal2nal
                # str(nucfile)                        # This part passes the string name of the current nucleotide fasta file as an arg to pal2nal
                # "-nogap"                            # This part passes the option -nogap to the pal2nal executable as specified in the paper,
                #                                     #     in order to remove all gapped positions from the resulting codon alignment
                # "-output", "fasta"                  # This part specifies fasta format for the output
            
        with open(outfile, "w") as handle:            # specify output file path as save destination
            handle.write(stdout)                      # save stdout to output file path

# UNCOMMENT the BELOW LINES to ERROR-CHECK
# pal2nal can be tricky
#        outerr = nucfolder / (nucname + ".err")
#        stderr = subprocess.run(["perl", str(pal2nalExec), str(pepfile), str(nucfile), "-nogap", "-output", "fasta"], capture_output=True).stderr.decode("utf-8")
#        with open(outerr, "w") as handle:             # specify error file path as save destination
#            handle.write(stderr)                      # save stderr to output file path
            
        continue
    else:
        continue

## STEP 3: Run codon alignments in IQ-TREE 
### Subject to maximum likelihood analysis with 10,000 Ultrafast bootstrap replicates
##### (Note: you can modify the tree output suffix at: treefix = ".[enter suffix here]")

In [355]:
# Store alignment suffix as a variable
nucalnfix = (nucfix + alnfix)
iqtrfix = ".contree"                                 # Store IQ-TREE's internal contree suffix as variable for naming purposes
treefix = ".tree"

# Loop over all files in nucleotide folder
for filename in os.listdir(nucfolder):               # specify the nucleotide folder as the one to look for files in
    if filename.endswith(nucalnfix):                 # make sure that only nucleotide alignments are used
        
        infile = nucfolder / filename                # specify full input nucleotide alignment path
        iqtfile = nucfolder / (filename + iqtrfix)   # specify IQ-TREE output tree path
        outfile = treefolder / (filename + treefix)  # specify full output tree path
        
        stdout = subprocess.run([str(IQExec), "-s", str(infile), "-nt", "AUTO", "-m", "MFP", "-bb", "10000"], capture_output=True).stdout.decode("utf-8")
                                                     # generate and run command, save as stdout
                #         COMMAND CONTEXT STRUCTURE: #
                # subprocess.run(...)                # This part tells python it's going to run some command-line-style command
                # ([command], ...)                   # This part specifies the command as a list of arguments
                # (..., capture_output=True)         # This part stores the output as bytes in the stdout built-in field of the subprocess module
                # ...(...).stdout...                 # This part retrieves that byte output from subprocess' stdout field
                # ...(...)...decode("utf-8")         # This part decodes the byte output into a string
                # stdout = subprocess.run(...)...    # This part stores that string as a python variable
                #                 COMMAND STRUCTURE: # 
                # str(IQExec)                        # This part tells python it's going to be running the IQ-TREE executable
                # "-s", str(infile)                  # This part passes option -s and the string name of the input file as args to IQ-TREE
                # "-m ..."                           # This part passes option -m to specify which substitution model will be used for the tree
                # "... MFP ..."                      # This part specifies using ModelFinder to determine the best-fit model, as in the paper
                # "... -bb 10000"                    # This part passes option -bb to specify the 10,000 Ultrafast bootstrap replicates should 
                #                                    #     be done, as specified in the paper
                
                # IQ-TREE takes care of saving the files, saves them in the same location as the input file
        
        copyfile(iqtfile, outfile)                   # This part copies the output file that IQ-TREE generates to your tree folder for analysis
        
        continue
    else:
        continue

## STEP 4: Control to fit the evolutionary frame of species
### No procedures were given for this step in the paper. 
##### Given the sensitivity of this analysis to rooting, manual rerooting seems likeliest, and this is what I did.
#### If in a later day validated procedures exist for rooting a gene tree with respect to known species trees, these should be used here.

In [356]:
# Store rooting suffix as a variable
rootfix = ".rooted"

## STEP 5: Test branch for selection 
### Test uses the branch-site model A in codeml in PAML

In [358]:
# Store rooted tree suffix as a variable
roottreefix = (nucalnfix + treefix + rootfix)           

outfix = ".txt"                                         # assigns output prefix to variable

# Loop over all files in tree folder
for filename in os.listdir(treefolder):                 # specify the tree folder as the one to look for files in
    if filename.endswith(roottreefix):                  # make sure that only rooted trees are used
        
        nucname = filename.replace(roottreefix, nucalnfix)   # generate the basename of peptide alignment's corresponding nucleotide fasta
        outname = (filename + outfix)                        # generate output filename
        seqfile = nucfolder / nucname                        # get path of sequence file
        treefile = treefolder / filename                     # get path of tree file
        outfileNull = codemlfolder / (outname + ".null")     # get path of null hypothesis output file
        outfileAlt = codemlfolder / (outname + ".alt")       # get path of alt hypothesis output file
        
        codemlCtlNull = codemlfolder / (codemlCtl + ".null") # generate codeml control file names for our null hypothesis as per paper
        codemlCtlAlt = codemlfolder / (codemlCtl + ".alt")   # generate codeml control file names for our alt hypothesis as per paper
        
        copyfile(PAMLcodemlCtl, codemlCtlNull)               # Adds fresh copies of the PAML codeml control file to the working directory
        copyfile(PAMLcodemlCtl, codemlCtlAlt)                #     so that editors below can function correctly for the current tree file
        
        
        ############################################################## SETUP for CODEML CONTROL FILE for NULL HYPOTHESIS
        with fileinput.FileInput(codemlCtlNull, inplace=True) as file:        # Specifies null hypothesis control file for editing
            for line in file:                                                 # iterates over lines
                print(line.replace("stewart.aa", str(seqfile)), end='')       # Replaces seqfile field with peptide sequence file path
        with fileinput.FileInput(codemlCtlNull, inplace=True) as file:        # Specifies null hypothesis control file for editing
            for line in file:                                                 # iterates over lines
                print(line.replace("stewart.trees", str(treefile)), end='')   # Replaces treefile field with tree file path
        with fileinput.FileInput(codemlCtlNull, inplace=True) as file:        # Specifies null hypothesis control file for editing 
            for line in file:                                                 # iterates over lines
                print(line.replace("mlc", str(outfileNull)), end='')          # Replaces outfile field with output file path
        with fileinput.FileInput(codemlCtlNull, inplace=True) as file:        # Specifies null hypothesis control file for editing 
            for line in file:                                                 # iterates over lines
                print(line.replace("seqtype = 2", "seqtype = 1"), end='')     # Replaces seqtype field with codon numeral as inferred from error
        with fileinput.FileInput(codemlCtlNull, inplace=True) as file:        # Specifies null hypothesis control file for editing 
            for line in file:                                                 # iterates over lines
                print(line.replace("NSsites = 0", "NSsites = 2"), end='')     # Changes NSsites field to value specified in paper
        with fileinput.FileInput(codemlCtlNull, inplace=True) as file:        # Specifies null hypothesis control file for editing 
            for line in file:                                                 # iterates over lines
                print(line.replace("fix_omega = 0", "fix_omega = 1"), end='') # Changes fix_omega field to value specified in paper
        with fileinput.FileInput(codemlCtlNull, inplace=True) as file:        # Specifies null hypothesis control file for editing 
            for line in file:                                                 # iterates over lines
                print(line.replace("omega = .4", "omega = 1"), end='')        # Changes omega field to value specified in paper
                
        ############################################################## SETUP for CODEML CONTROL FILE for ALT HYPOTHESIS
        with fileinput.FileInput(codemlCtlAlt, inplace=True) as file:         # Specifies alt hypothesis control file for editing
            for line in file:                                                 # iterates over lines
                print(line.replace("stewart.aa", str(seqfile)), end='')       # Replaces seqfile field with peptide sequence file path
        with fileinput.FileInput(codemlCtlAlt, inplace=True) as file:         # Specifies alt hypothesis control file for editing
            for line in file:                                                 # iterates over lines
                print(line.replace("stewart.trees", str(treefile)), end='')   # Replaces treefile field with tree file path
        with fileinput.FileInput(codemlCtlAlt, inplace=True) as file:         # Specifies alt hypothesis control file for editing 
            for line in file:                                                 # iterates over lines
                print(line.replace("mlc", str(outfileAlt)), end='')           # Replaces outfile field with output file path
        with fileinput.FileInput(codemlCtlAlt, inplace=True) as file:         # Specifies null hypothesis control file for editing 
            for line in file:                                                 # iterates over lines
                print(line.replace("seqtype = 2", "seqtype = 1"), end='')     # Replaces seqtype field with codon numeral as inferred from paper
        with fileinput.FileInput(codemlCtlAlt, inplace=True) as file:         # Specifies alt hypothesis control file for editing 
            for line in file:                                                 # iterates over lines
                print(line.replace("NSsites = 0", "NSsites = 2"), end='')     # Changes NSsites field to value specified in paper
        with fileinput.FileInput(codemlCtlAlt, inplace=True) as file:         # Specifies alt hypothesis control file for editing 
            for line in file:                                                 # iterates over lines
                print(line.replace("fix_omega = 0", "fix_omega = 0"), end='') # Changes fix_omega field to value specified in paper
        with fileinput.FileInput(codemlCtlAlt, inplace=True) as file:         # Specifies alt hypothesis control file for editing 
            for line in file:                                                 # iterates over lines
                print(line.replace("omega = .4", "omega = 1.5"), end='')      # Changes omega field to value specified in paper
        
        
        ############################### RUN COMMAND for CODEML for NULL HYPOTHESIS
        copyfile(codemlCtlNull, codemlCtl)           # This part copies in the correct modified control file into the control file name 
                #                                    #     expected by codeml, the one for the null hypothesis
        stdout = subprocess.run([str(PAMLExec)], capture_output=True).stdout.decode("utf-8")
                                                     # generate and run command, save as stdout
                #         COMMAND CONTEXT STRUCTURE: #
                # subprocess.run(...)                # This part tells python it's going to run some command-line-style command
                # ([command], ...)                   # This part specifies the command as a list of arguments
                # (..., capture_output=True)         # This part stores the output as bytes in the stdout built-in field of the subprocess module
                # ...(...).stdout...                 # This part retrieves that byte output from subprocess' stdout field
                # ...(...)...decode("utf-8")         # This part decodes the byte output into a string
                # stdout = subprocess.run(...)...    # This part stores that string as a python variable
                #                 COMMAND STRUCTURE: # 
                # str(PAMLExec)                      # This part tells python it's going to be running the IQ-TREE executable
                #                                    #     which is all that's needed after changing the control file
                
                # PAML takes care of saving the output file

# UNCOMMENT the BELOW LINES to ERROR-CHECK
# PAML can be tricky
#        outerr = codemlfolder / (outname + ".null.err")
#        stderr = subprocess.run([str(PAMLExec)], capture_output=True).stderr.decode("utf-8")
#        with open(outerr, "w") as handle:             # specify error file path as save destination
#            handle.write(stderr)                      # save stderr to output file path
      
    
        ############################### RUN COMMAND for CODEML for ALT HYPOTHESIS
        copyfile(codemlCtlAlt, codemlCtl)            # This part copies in the correct modified control file into the control file name 
                #                                    #     expected by codeml, the one for the alt hypothesis
        stdout = subprocess.run([str(PAMLExec)], capture_output=True).stdout.decode("utf-8")
                                                     # generate and run command, save as stdout
                #         COMMAND CONTEXT STRUCTURE: #
                # subprocess.run(...)                # This part tells python it's going to run some command-line-style command
                # ([command], ...)                   # This part specifies the command as a list of arguments
                # (..., capture_output=True)         # This part stores the output as bytes in the stdout built-in field of the subprocess module
                # ...(...).stdout...                 # This part retrieves that byte output from subprocess' stdout field
                # ...(...)...decode("utf-8")         # This part decodes the byte output into a string
                # stdout = subprocess.run(...)...    # This part stores that string as a python variable
                #                 COMMAND STRUCTURE: # 
                # str(PAMLExec)                      # This part tells python it's going to be running the IQ-TREE executable
                #                                    #     which is all that's needed after changing the control file
                
                # PAML takes care of saving the output file
                
# UNCOMMENT the BELOW LINES to ERROR-CHECK
# PAML can be tricky                
#        outerr = codemlfolder / (outname + ".alt.err")
#        stderr = subprocess.run([str(PAMLExec)], capture_output=True).stderr.decode("utf-8")
#        with open(outerr, "w") as handle:             # specify error file path as save destination
#            handle.write(stderr)                      # save stderr to output file path 

        continue
    else:
        continue

# Workspace cleanup after codeml
os.remove("rst")
os.remove("rst1")
os.remove("rub")
os.remove("codeml.ctl")
os.remove(codemlCtlNull)
os.remove(codemlCtlAlt)
os.remove("2NG.dN")
os.remove("2NG.dS")
os.remove("2NG.t")
os.remove("4fold.nuc")